<a href="https://colab.research.google.com/github/Atik-Ahamed/Semantic-Segmentation-NODE/blob/master/sem_seg_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary file

In [ ]:
import torch.nn
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm
from PIL import Image
import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F
from torch.autograd import Variable
from torchvision import models
from torchsummary import summary
use_cuda = torch.cuda.is_available()
import cv2
import random
import os
import torchvision.transforms as T
from PIL import Image
import torchvision.transforms.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import pandas as pd
from sklearn.model_selection import train_test_split

# downloading from git

In [ ]:
!git clone https://github.com/usuyama/pytorch-unet.git

In [ ]:
!mv  -v /content/pytorch-unet/* /content/

In [ ]:
!rm -r /content/pytorch-unet

# generating dataset

In [ ]:
input_shape=(192,192,3)
#here in the pytorch model channel first is used it is changed as required in the model definition
#but to convention here the channel last order is used as input shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import helper
import simulation

# Generate some random images
input_images, target_masks = simulation.generate_random_data(input_shape[0], input_shape[1], count=3)

for x in [input_images, target_masks]:
    print(x.shape)
    print(x.min(), x.max())

# Change channel-order and make 3 channels for matplot
input_images_rgb = [x.astype(np.uint8) for x in input_images]

# Map each channel (i.e. class) to each color
target_masks_rgb = [helper.masks_to_colorimg(x) for x in target_masks]

# Left: Input image (black and white), Right: Target mask (6ch)
helper.plot_side_by_side([input_images_rgb, target_masks_rgb])

# Dataloader

In [ ]:
class SimDataset(Dataset):
    def __init__(self, count, transform=None):
        #here i need extra param as length and type as train or val
        self.input_images, self.target_masks = simulation.generate_random_data(input_shape[0], input_shape[1], count=count)
        self.transform = transform

    def __len__(self):
        #return self.length
        return len(self.input_images)

    def __getitem__(self, idx):
      #here i need to make change
        #get_data if train or val
        image = self.input_images[idx]
        mask = self.target_masks[idx]
        if self.transform:
            image = self.transform(image)

        return [image, mask]

# use the same transformations for train/val in this example
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])

train_set = SimDataset(500, transform = trans)
val_set = SimDataset(50, transform = trans)

image_datasets = {
    'train': train_set, 'val': val_set
}

batch_size = 4

dataloaders = {
    'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)
}

In [ ]:
import torchvision.utils

def reverse_transform(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    inp = (inp * 255).astype(np.uint8)

    return inp

# Get a batch of training data
inputs, masks = next(iter(dataloaders['train']))

print(inputs.shape, masks.shape)

plt.imshow(reverse_transform(inputs[3]))

# ResNET UNET author's model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )


class ResNetUNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()

        self.base_model = models.resnet18(pretrained=False)
        self.base_layers = list(self.base_model.children())

        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

        self.conv_last = nn.Conv2d(64, n_class, 1)

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)

        layer0 = self.layer0(input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)

        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)

        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)

        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)

        out = self.conv_last(x)

        return out

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetUNet(n_class=6)
model = model.to(device)

# check keras-like model summary using torchsummary
from torchsummary import summary
summary(model, input_size=(3, 192, 192))

# Training Resnet Author's model

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

from collections import defaultdict
import torch.nn.functional as F
from loss import dice_loss
loss1=nn.CrossEntropyLoss()
sig=nn.Sigmoid()

def calc_metrics(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    numerator =  (pred * target).sum((1, 2, 3))
    denominator = (pred + target).sum((1, 2, 3))

    iou=(numerator / (denominator-numerator)).mean()


    # pred1=pred.data.cpu().numpy()
    # target1=target.data.cpu().numpy()
    # pred1=np.argmax(pred1,axis=1)
    # target1=np.argmax(target1,axis=1)
    # #print(pred1.shape)
    # pred1=np.reshape(pred1,(pred1.shape[0]*pred1.shape[1]*pred1.shape[2]))
    # target1=np.reshape(target1,(target1.shape[0]*target1.shape[1]*target1.shape[2]))
    # same=0
    # #print(pred.shape)
    # for i in range(0,pred1.shape[0]):
    #   if pred1[i]==target1[i]:
    #     same=same+1
    # acc=same/pred1.shape[0]
    #print(acc)

    # _, preds = torch.max(pred, dim=1)
    # _, targets = torch.max(target, dim=1)
    # valid = (targets >= 0).long()
    # acc_sum = torch.sum(valid * (preds == targets).long())
    # pixel_sum = torch.sum(valid)
    # acc = acc_sum.float() / (pixel_sum.float() + 1e-10)


    #acc=(torch.sum((torch.argmax(pred,dim=1)==torch.argmax(target,dim=1))).item())/(target.shape[0]*target.shape[1]*target.shape[2]*target.shape[3])
    #print(acc)

    #print(target.size(0))
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] += iou.data.cpu().numpy() * target.size(0)
   # metrics['acc'] +=acc* target.size(0)

    return loss

 
def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    best_iou=-1
    best_dice=1e10
    best_bce=1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(inputs.shape,labels.shape)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #print(outputs.shape)
                    loss = calc_metrics(outputs, labels, metrics)
                    #print(loss.item())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_dice = metrics['dice'] / epoch_samples
            epoch_bce = metrics['bce'] / epoch_samples
            epoch_iou = metrics['iou'] / epoch_samples

            # deep copy the model
            if phase == 'val':
              if epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
              if epoch_dice<best_dice:
                best_dice=epoch_dice
              if epoch_bce<best_bce:
                best_bce=epoch_bce
              if epoch_iou>best_iou:
                best_iou=epoch_iou

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val dice: {:4f}'.format(best_dice))
    print('Best val bce: {:4f}'.format(best_bce))
    print('Best val iou: {:4f}'.format(best_iou))
     # load best model weights
    model.load_state_dict(best_model_wts)
    return model




device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=200, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=500)

# ODE model our created model

In [ ]:
def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z
  
  
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), (z, t) + tuple(self.parameters()), grad_outputs=(a),
            allow_unused=True, retain_graph=True
        )
        # grad method automatically sums gradients for batch items, we have to expand them back 
        if adfdp is not None:
           adfdp = torch.cat([p_grad.flatten() for p_grad in adfdp]).unsqueeze(0)
           adfdp = adfdp.expand(batch_size, -1) / batch_size
        if adfdt is not None:
           adfdt = adfdt.expand(batch_size, 1) / batch_size
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters)      
      
class ODEAdjoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z0, t, flat_parameters, func):
        assert isinstance(func, ODEF)
        bs, *z_shape = z0.size()
        time_len = t.size(0)

        with torch.no_grad():
            z = torch.zeros(time_len, bs, *z_shape).to(z0)
            z[0] = z0
            for i_t in range(time_len - 1):
                z0 = ode_solve(z0, t[i_t], t[i_t+1], func)
                z[i_t+1] = z0

        ctx.func = func
        ctx.save_for_backward(t, z.clone(), flat_parameters)
        return z

    @staticmethod
    def backward(ctx, dLdz):
        """
        dLdz shape: time_len, batch_size, *z_shape
        """
        func = ctx.func
        t, z, flat_parameters = ctx.saved_tensors
        time_len, bs, *z_shape = z.size()
        n_dim = np.prod(z_shape)
        n_params = flat_parameters.size(0)

        # Dynamics of augmented system to be calculated backwards in time
        def augmented_dynamics(aug_z_i, t_i):
            """
            tensors here are temporal slices
            t_i - is tensor with size: bs, 1
            aug_z_i - is tensor with size: bs, n_dim*2 + n_params + 1
            """
            z_i, a = aug_z_i[:, :n_dim], aug_z_i[:, n_dim:2*n_dim]  # ignore parameters and time

            # Unflatten z and a
            z_i = z_i.view(bs, *z_shape)
            a = a.view(bs, *z_shape)
            with torch.set_grad_enabled(True):
                t_i = t_i.detach().requires_grad_(True)
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, grad_outputs=a)  # bs, *z_shape
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(bs, *z_shape).to(z_i)
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(bs, n_params).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(bs, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(bs, n_dim)
            adfdz = adfdz.view(bs, n_dim) 
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)

        dLdz = dLdz.view(time_len, bs, n_dim)  # flatten dLdz for convenience
        with torch.no_grad():
            ## Create placeholders for output gradients
            # Prev computed backwards adjoints to be adjusted by direct gradients
            adj_z = torch.zeros(bs, n_dim).to(dLdz)
            adj_p = torch.zeros(bs, n_params).to(dLdz)
            # In contrast to z and p we need to return gradients for all times
            adj_t = torch.zeros(time_len, bs, 1).to(dLdz)

            for i_t in range(time_len-1, 0, -1):
                z_i = z[i_t]
                t_i = t[i_t]
                f_i = func(z_i, t_i).view(bs, n_dim)

                # Compute direct gradients
                dLdz_i = dLdz[i_t]
                dLdt_i = torch.bmm(torch.transpose(dLdz_i.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

                # Adjusting adjoints with direct gradients
                adj_z += dLdz_i
                adj_t[i_t] = adj_t[i_t] - dLdt_i

                # Pack augmented variable
                aug_z = torch.cat((z_i.view(bs, n_dim), adj_z, torch.zeros(bs, n_params).to(z), adj_t[i_t]), dim=-1)

                # Solve augmented system backwards
                aug_ans = ode_solve(aug_z, t_i, t[i_t-1], augmented_dynamics)

                # Unpack solved backwards augmented system
                adj_z[:] = aug_ans[:, n_dim:2*n_dim]
                adj_p[:] += aug_ans[:, 2*n_dim:2*n_dim + n_params]
                adj_t[i_t-1] = aug_ans[:, 2*n_dim + n_params:]

                del aug_z, aug_ans

            ## Adjust 0 time adjoint with direct gradients
            # Compute direct gradients 
            dLdz_0 = dLdz[0]
            dLdt_0 = torch.bmm(torch.transpose(dLdz_0.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

            # Adjust adjoints
            adj_z += dLdz_0
            adj_t[0] = adj_t[0] - dLdt_0
        return adj_z.view(bs, *z_shape), adj_t, adj_p, None

 
class NeuralODE(nn.Module):
    def __init__(self, func):
        super(NeuralODE, self).__init__()
        assert isinstance(func, ODEF)
        self.func = func

    def forward(self, z0, t=Tensor([0., 1.]), return_whole_sequence=False):
        t = t.to(z0)
        z = ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)
        if return_whole_sequence:
            return z
        else:
            return z[-1]

  
def add_time(in_tensor, t):
    bs, c, w, h = in_tensor.shape
    return torch.cat((in_tensor, t.expand(bs, 1, w, h)), dim=1)
  
  
  
class ConvODEF(ODEF):
    def __init__(self, dim):
        super(ConvODEF, self).__init__()

        self.kernel_size=3
          
 
        self.conv2d=torch.nn.Conv2d(kernel_size=3, in_channels=dim+1, out_channels=dim,padding=1)
        self.bn=torch.nn.BatchNorm2d(dim)

        


    def forward(self, x, t):
        xt = add_time(x, t)
        #print('shape=',xt.shape)
        h = self.bn(torch.relu(self.conv2d(xt)))
        ht = add_time(h, t)
        dxdt = self.bn(torch.relu(self.conv2d(ht)))
        return dxdt
      


In [ ]:
func = ConvODEF(64)
#print(func)
ode = NeuralODE(func)
#print(ode)

In [ ]:
class SEM_SEG_ODE(nn.Module):
    def __init__(self, ode,n_class,input_shape):
        super(SEM_SEG_ODE, self).__init__()
        self.feature = ode
        self.input_shape=input_shape[1]
        self.n_filters=16
        self.dropout=.05
        self.batchnorm=True
        self.kernel_size=3
        self.n_class=n_class
          
        self.relu=torch.nn.ReLU(inplace=True)
        self.maxpool=torch.nn.MaxPool2d(2)
        self.dropout=torch.nn.Dropout2d(.05)
        self.conv2d=torch.nn.Conv2d(kernel_size=3, in_channels=input_shape[2], out_channels=16,padding=1)
        self.bn=torch.nn.BatchNorm2d(16)

        self.conv2d_1=torch.nn.Conv2d(kernel_size=3, in_channels=16, out_channels=32,padding=1)
        self.bn1=torch.nn.BatchNorm2d(32)
        
        self.conv2d_2=torch.nn.Conv2d(kernel_size=3, in_channels=32, out_channels=64,padding=1)
        self.bn2=torch.nn.BatchNorm2d(64)
        
        self.conv2d_3=torch.nn.Conv2d(kernel_size=3, in_channels=64, out_channels=128,padding=1)
        self.bn3=torch.nn.BatchNorm2d(128)

        self.conv2d_4=torch.nn.Conv2d(kernel_size=3, in_channels=128, out_channels=256,padding=1)
        self.bn4=torch.nn.BatchNorm2d(256)

        self.conv2d_5=torch.nn.Conv2d(kernel_size=3, in_channels=256, out_channels=128,padding=1)
        self.bn5=torch.nn.BatchNorm2d(128)

        self.conv2d_6=torch.nn.Conv2d(kernel_size=3, in_channels=128, out_channels=64,padding=1)
        self.bn6=torch.nn.BatchNorm2d(64)

        self.conv2d_7=torch.nn.Conv2d(kernel_size=3, in_channels=64, out_channels=32,padding=1)
        self.bn7=torch.nn.BatchNorm2d(32)

        self.conv2d_8=torch.nn.Conv2d(kernel_size=3, in_channels=32, out_channels=16,padding=1)
        self.bn8=torch.nn.BatchNorm2d(16)

        self.up=torch.nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up1=torch.nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up2=torch.nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up3=torch.nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.conv2d_9=torch.nn.Conv2d(kernel_size=1, in_channels=16, out_channels=self.n_class,padding=0)
        self.sig=torch.nn.Sigmoid()

    def forward(self, x):


        c1=self.conv2d(x)
        if self.batchnorm:
          c1=self.bn(c1)
        c1=self.relu(c1)

        p1=self.maxpool(c1)
        p1=self.dropout(p1)

        c2=self.conv2d_1(p1)
        if self.batchnorm:
          c2=self.bn1(c2)
        c2=self.relu(c2)

        p2=self.maxpool(c2)
        p2=self.dropout(p2)

        c3=self.conv2d_2(p2)
        if self.batchnorm:
          c3=self.bn2(c3)
        c3=self.relu(c3)

        p3=self.maxpool(c3)
        p3=self.dropout(p3)

        ode_here=self.feature(p3)#64-64


        c4=self.conv2d_3(ode_here)#64-128
        if self.batchnorm:
          c4=self.bn3(c4)
        c4=self.relu(c4)

        p4=self.maxpool(c4)
        p4=self.dropout(p4)
        
        c5=self.conv2d_4(p4)#128-256
        if self.batchnorm:
          c5=self.bn4(c5)
        c5=self.relu(c5)
        
        u6=self.up(c5)#256-128
        u6=torch.cat((u6,c4),1)
        u6=self.dropout(u6)

        c6=self.conv2d_5(u6)
        if self.batchnorm:
          c6=self.bn5(c6)
        c6=self.relu(c6)

        u7=self.up1(c6)#128-64
        u7=torch.cat((u7,c3),1)
        u7=self.dropout(u7)
        

        c7=self.conv2d_6(u7)#64-32
        if self.batchnorm:
          c7=self.bn6(c7)
        c7=self.relu(c7)

        u8=self.up2(c7)
        u8=torch.cat((u8,c2),1)
        u8=self.dropout(u8)
        

        c8=self.conv2d_7(u8)
        if self.batchnorm:
          c8=self.bn7(c8)
        c8=self.relu(c8)

        u9=self.up3(c8)
        u9=torch.cat((u9,c1),1)
        u9=self.dropout(u9)
        

        c9=self.conv2d_8(u9)
        if self.batchnorm:
          c9=self.bn8(c9)
        c9=self.relu(c9)

        output=self.conv2d_9(c9)
        #output=self.sig(output)

        return output

  

In [ ]:
model = SEM_SEG_ODE(ode,n_class=6,input_shape=(192,192,3))
if use_cuda:
    model = model.cuda()

summary(model, (3, 192, 192))

# defining metrics function

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
from loss import dice_loss
loss1=nn.CrossEntropyLoss()
sig=nn.Sigmoid()

def calc_metrics(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    numerator =  (pred * target).sum((1, 2, 3))
    denominator = (pred + target).sum((1, 2, 3))

    iou=(numerator / (denominator-numerator)).mean()


    # pred1=pred.data.cpu().numpy()
    # target1=target.data.cpu().numpy()
    # pred1=np.argmax(pred1,axis=1)
    # target1=np.argmax(target1,axis=1)
    # #print(pred1.shape)
    # pred1=np.reshape(pred1,(pred1.shape[0]*pred1.shape[1]*pred1.shape[2]))
    # target1=np.reshape(target1,(target1.shape[0]*target1.shape[1]*target1.shape[2]))
    # same=0
    # #print(pred.shape)
    # for i in range(0,pred1.shape[0]):
    #   if pred1[i]==target1[i]:
    #     same=same+1
    # acc=same/pred1.shape[0]
    #print(acc)

    # _, preds = torch.max(pred, dim=1)
    # _, targets = torch.max(target, dim=1)
    # valid = (targets >= 0).long()
    # acc_sum = torch.sum(valid * (preds == targets).long())
    # pixel_sum = torch.sum(valid)
    # acc = acc_sum.float() / (pixel_sum.float() + 1e-10)


    #acc=(torch.sum((torch.argmax(pred,dim=1)==torch.argmax(target,dim=1))).item())/(target.shape[0]*target.shape[1]*target.shape[2]*target.shape[3])
    #print(acc)

    #print(target.size(0))
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] += iou.data.cpu().numpy() * target.size(0)
   # metrics['acc'] +=acc* target.size(0)

    return loss

# Defining training functions

In [ ]:

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    best_iou=-1
    best_dice=1e10
    best_bce=1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(inputs.shape,labels.shape)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #print(outputs.shape)
                    loss = calc_metrics(outputs, labels, metrics)
                    #print(loss.item())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_dice = metrics['dice'] / epoch_samples
            epoch_bce = metrics['bce'] / epoch_samples
            epoch_iou = metrics['iou'] / epoch_samples

            # deep copy the model
            if phase == 'val':
              if epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
              if epoch_dice<best_dice:
                best_dice=epoch_dice
              if epoch_bce<best_bce:
                best_bce=epoch_bce
              if epoch_iou>best_iou:
                best_iou=epoch_iou

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val dice: {:4f}'.format(best_dice))
    print('Best val bce: {:4f}'.format(best_bce))
    print('Best val iou: {:4f}'.format(best_iou))
     # load best model weights
    model.load_state_dict(best_model_wts)
    return model



# training the unet ode model

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=200, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=500)

# making prediction on test data

In [ ]:
import math
def pred_small_dataset(my_model):

    my_model.eval()   # Set model to the evaluation mode

    # Create another simulation dataset for test
    test_dataset = SimDataset(3, transform = trans)
    test_loader = DataLoader(test_dataset, batch_size=3, shuffle=False, num_workers=0)

    # Get the first batch
    inputs, labels = next(iter(test_loader))
    inputs = inputs.to(device)
    labels = labels.to(device)

    # Predict
    pred = model(inputs)
    # The loss functions include the sigmoid function.
    pred = F.sigmoid(pred)
    #print(torch.sum(pred,dim=1))
    pred = pred.data.cpu().numpy()
    #print(pred.shape)

    # Change channel-order and make 3 channels for matplot
    input_images_rgb = [reverse_transform(x) for x in inputs.cpu()]

    # Map each channel (i.e. class) to each color
    target_masks_rgb = [helper.masks_to_colorimg(x) for x in labels.cpu().numpy()]
    pred_rgb = [helper.masks_to_colorimg(x) for x in pred]

    helper.plot_side_by_side([input_images_rgb, target_masks_rgb, pred_rgb])

In [ ]:
pred_small_dataset(model)

# downloading data from kaggle neuclie segmentation dataset

In [ ]:
from google.colab import files
files.upload()

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c data-science-bowl-2018

!unzip -q '/content/stage1_sample_submission.csv.zip' -d '/content/kaggle_neuclei'
!unzip -q '/content/stage1_solution.csv.zip' -d '/content/kaggle_neuclei'
!unzip -q '/content/stage1_test.zip' -d '/content/kaggle_neuclei/stage1_test'
!unzip -q '/content/stage1_train.zip' -d '/content/kaggle_neuclei/stage1_train'
!unzip -q '/content/stage1_train_labels.csv.zip' -d '/content/kaggle_neuclei'
!unzip -q '/content/stage2_sample_submission_final.csv.zip' -d '/content/kaggle_neuclei'
!unzip -q '/content/stage2_test_final.zip' -d '/content/kaggle_neuclei/stage2_test_final'

!rm '/content/stage1_sample_submission.csv.zip'
!rm '/content/stage1_solution.csv.zip'
!rm '/content/stage1_test.zip'
!rm '/content/stage1_train.zip'
!rm '/content/stage1_train_labels.csv.zip'
!rm '/content/stage2_sample_submission_final.csv.zip'
!rm '/content/stage2_test_final.zip'



# Dataloader for neuclie segmentation

In [ ]:
train_ids=os.listdir('/content/kaggle_neuclei/stage1_train')
#train_ids=train_ids[:int(len(train_ids)*.20)]
train_ids,val_ids=train_test_split(train_ids,test_size=.1,random_state=42)
print(len(train_ids))
print(len(val_ids))

In [ ]:
train_img=cv2.imread('/content/kaggle_neuclei/stage1_train/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552/images/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552.png')
plt.imshow(train_img)
plt.show()
plt.figure()

mask_img=np.zeros(shape=(train_img.shape[0],train_img.shape[1],3))

mask_one_file=os.listdir('/content/kaggle_neuclei/stage1_train/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552/masks')
for m in mask_one_file:
  single_mask=cv2.imread('/content/kaggle_neuclei/stage1_train/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552/masks/'+m)
  mask_img=np.maximum(mask_img,single_mask)
  # for i in range(0,train_img.shape[0]):
  #   for j in range(0,train_img.shape[1]):
  #     mask_img[i,j]=single_mask[i,j][0]

plt.imshow(mask_img)
plt.show()

In [ ]:
class Rescale:
    def __init__(self, output_size: int):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image: np.ndarray) -> np.ndarray:
        return cv2.resize(image, (self.output_size, self.output_size), cv2.INTER_AREA)

In [ ]:
class NeucleiDataLoader(Dataset):
    def __init__(self, img_ids,path_pref,im_shape, transform=None,mask_transform=None):
        self.transform = transform
        self.img_ids=img_ids
        self.path_pref=path_pref#'/content/kaggle_neuclei/stage1_train/'
        self.im_shape=im_shape
        self.mask_transform=mask_transform

    def __len__(self):      
        return len(self.img_ids)

    def __getitem__(self, idx):      
        image = cv2.imread(self.path_pref+self.img_ids[idx]+'/images/'+self.img_ids[idx]+'.png')
        target_img = np.zeros(image.shape[:2], dtype=np.uint8)
        mask_one_file=os.listdir(self.path_pref+self.img_ids[idx]+'/masks')
        for m in mask_one_file:
            single_mask=cv2.imread(self.path_pref+self.img_ids[idx]+'/masks/'+m,0)
            #print(single_mask.shape)
            target_img=np.maximum(target_img,single_mask)
        
        if self.transform:
          image=self.transform(image)
        if self.mask_transform:
          target_img=self.mask_transform(target_img)

        return [image, target_img]


trans = transforms.Compose([
    Rescale(256),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])


train_set = NeucleiDataLoader(train_ids,'/content/kaggle_neuclei/stage1_train/',256,trans,trans)
val_set = NeucleiDataLoader(val_ids,'/content/kaggle_neuclei/stage1_train/',256,trans,trans)

image_datasets = {
    'train': train_set, 'val': val_set
}

batch_size = 4

neuclie_dataloaders = {
    'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)
}

**Testing the dataloader**

In [ ]:
image,mask=next(iter(neuclie_dataloaders['val']))
print(image.shape,mask.shape)
from torchvision.transforms import transforms, ToTensor, ToPILImage
def reverse_transform(inp,mask):   
    output_transforms = transforms.Compose([
            ToPILImage()])
    mask=output_transforms(mask) 
    inp=output_transforms(inp)
   
    return inp,mask

image,mask=reverse_transform(image[0],mask[0])
plt.imshow(image)
plt.title('input image')
plt.figure()
plt.imshow(mask,cmap='gray')
plt.title('target mask')


# Defining the model

In [ ]:
model = SEM_SEG_ODE(ode,n_class=1,input_shape=(256,256,3))
if use_cuda:
    model = model.cuda()

summary(model, (3, 256, 256))

# defining the metrics function

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
from loss import dice_loss
ce=nn.CrossEntropyLoss()
def dice_loss1(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Soft dice loss = 2*|A∩B| / |A|+|B|
    """
    numerator = 2 * (input * target).sum((1, 2, 3))
    denominator = (input + target).sum((1, 2, 3))

    return (1 - numerator / denominator).mean()


    
def calc_metrics(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss1(pred, target)

    loss = bce  + dice #* (1 - bce_weight)

    numerator =  (pred * target).sum((1, 2, 3))
    denominator = (pred + target).sum((1, 2, 3))

    iou=(numerator / (denominator-numerator)).mean()

    # pred1=pred.data.cpu().numpy()
    # target1=target.data.cpu().numpy()
    # #pred1=np.argmax(pred1,axis=1)
    # #target1=np.argmax(target1,axis=1)
    # #print(pred1.shape)
    # pred1=np.reshape(pred1,(pred1.shape[0]*pred1.shape[1]*pred1.shape[2]*pred1.shape[3]))
    # target1=np.reshape(target1,(target1.shape[0]*target1.shape[1]*target1.shape[2]*target1.shape[3]))
    # same=0
    #print(pred.shape)
    # for i in range(0,pred1.shape[0]):
    #   if pred1[i]>=.5:
    #     pred1[i]=1.0
    #   if pred1[i]<.5:
    #     pred1[i]=0.0
    #   if pred1[i]==target1[i]:
    #     same=same+1
    # acc=same/pred1.shape[0]
    # #print(acc)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] +=iou.data.cpu().numpy() * target.size(0)
    #metrics['acc'] +=acc * target.size(0)


    return loss

# def calc_ce_loss(pred,target,metrics):
#   target=target.data.cpu().numpy()
#   target=np.argmax(target,axis=1)
#   #print(target.shape)
#   target=torch.from_numpy(target)
#   target=target.cuda()
#   ce_loss=ce(pred,target)
#   #print(ce_loss.data)
#   metrics['ce']+=ce_loss.data.cpu().numpy()*target.size(0)
#   return ce_loss


# Defining training function

In [ ]:

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    best_iou=-1
    best_dice=1e10
    best_bce=1e10
    #best_dice_co=-1



    for epoch in range(num_epochs):
        for_once=1
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in neuclie_dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #inputs=inputs.type(torch.cuda.FloatTensor)
                #labels=labels.type(torch.cuda.FloatTensor)
                #print(inputs.shape,labels.shape)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # if for_once==1:

                    #   img,segmented_mask=reverse_transform(inputs[0],outputs[0])
                    #   img,mask=reverse_transform(inputs[0],labels[0])
                    #   plt.imshow(img)
                    #   plt.title('input image')
                    #   plt.figure()
                    #   plt.imshow(mask,cmap='gray')
                    #   plt.title('target mask')

                    #   plt.figure()
                    #   plt.imshow(segmented_mask,cmap='gray')
                    #   plt.title('predected mask')
                    #   for_once=0

                    #print(outputs.shape)
                    #outputs=outputs.int()
                    loss = calc_metrics(outputs, labels, metrics)

                    #print(loss.item())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_dice = metrics['dice'] / epoch_samples
            epoch_bce = metrics['bce'] / epoch_samples
            epoch_iou = metrics['iou'] / epoch_samples
            #epoch_dice_co=metrics['dice_co'] / epoch_samples

            # deep copy the model
            if phase == 'val':
              if epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
              if epoch_dice<best_dice:
                best_dice=epoch_dice
              if epoch_bce<best_bce:
                best_bce=epoch_bce
              if epoch_iou>best_iou:
                best_iou=epoch_iou
              # if epoch_dice_co>best_dice_co:
              #   best_dice_co=epoch_dice_co

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        #for_once=0

    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val dice: {:4f}'.format(best_dice))
    print('Best val bce: {:4f}'.format(best_bce))
    print('Best val iou: {:4f}'.format(best_iou))
    #print('Best val dice_co: {:4f}'.format(best_dice_co))
     # load best model weights
    model.load_state_dict(best_model_wts)
    return model



# Training the odeunet model

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=50)

# Training the fcn/unet model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fcn_unet_nuclei = ResNetUNet(n_class=1)
fcn_unet_nuclei = fcn_unet_nuclei.to(device)

# check keras-like model summary using torchsummary
from torchsummary import summary
print(summary(fcn_unet_nuclei, input_size=(3, 192, 192)))

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, fcn_unet_nuclei.parameters()), lr=1e-3)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

fcn_unet_nuclei = train_model(fcn_unet_nuclei, optimizer_ft, exp_lr_scheduler, num_epochs=50)

# Training Unet Buda

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return self.conv(dec1)

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )
buda_nuclei_model=UNet()

if use_cuda:
    buda_nuclei_model = buda_nuclei_model.cuda()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, buda_nuclei_model.parameters()), lr=1e-3)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
print(summary(buda_nuclei_model, (3, 256, 256)))
buda_nuclei_model = train_model(buda_nuclei_model, optimizer_ft, exp_lr_scheduler, num_epochs=50)


# Predicting on some data

In [ ]:
image,mask=next(iter(neuclie_dataloaders['val']))
print(mask.shape)
image=image.to(device)
mask=mask.to(device)
model=model.eval()
outputs=model(image)
outputs=F.sigmoid(outputs)
print(outputs.shape)

def reverse_transform(inp,mask):   
    output_transforms = transforms.Compose([
            ToPILImage()])
    mask=mask.cpu()
    inp=inp.cpu()
    mask=output_transforms(mask) 
    inp=output_transforms(inp)
   
    return inp,mask

img,segmented_mask=reverse_transform(image[0],outputs[0])
img,mask=reverse_transform(image[0],mask[0])
plt.imshow(img)
plt.title('input image')
plt.figure()
plt.imshow(mask,cmap='gray')
plt.title('target mask')

plt.figure()
plt.imshow(segmented_mask,cmap='gray')
plt.title('predected mask')


# predicting on test data

In [ ]:
model=model.eval()
test_single=cv2.imread('/content/kaggle_neuclei/stage1_test/0114f484a16c152baa2d82fdd43740880a762c93f436c8988ac461c5c9dbe7d5/images/0114f484a16c152baa2d82fdd43740880a762c93f436c8988ac461c5c9dbe7d5.png')
trans_test=transforms.Compose([
            Rescale(256),
            ToTensor()])
test_single=trans_test(test_single)

output = model(test_single.unsqueeze(0).cuda())  # Add batch dim
output_transforms = transforms.Compose([
            ToPILImage()])
output=F.sigmoid(output)
mask =output_transforms(output.squeeze(0).cpu())  # Remove batch dim




fig, axs = plt.subplots(nrows=2,ncols=2, sharex='all', sharey='all', figsize=( 10,  10))

# axs[0].imshow(x, origin='upper', interpolation=interp)

# axs[1].set_title('blue should be down')
# axs[1].imshow(x, origin='lower', interpolation=interp)
#plt.show()


#plt.figure(figsize=(15,10))
#plt.subplot(2,2,1)
#plt.imshow(output_transforms(test_single))
axs[0,0].imshow(output_transforms(test_single))
#plt.title('input test image')

#plt.subplot(2,2,2)
#plt.imshow(mask,cmap='gray')
axs[0,1].imshow(mask,cmap='gray')
#plt.title('output mask')

test_single=cv2.imread('/content/kaggle_neuclei/stage2_test_final/00b4b9c026cfc91af21691503bad67081fc02d5c77b3cd27200d02dca6966b83/images/00b4b9c026cfc91af21691503bad67081fc02d5c77b3cd27200d02dca6966b83.png')
trans_test=transforms.Compose([
            Rescale(256),
            ToTensor()])
test_single=trans_test(test_single)

output = model(test_single.unsqueeze(0).cuda())  # Add batch dim
output_transforms = transforms.Compose([
            ToPILImage()])
output=F.sigmoid(output)
mask =output_transforms(output.squeeze(0).cpu())  # Remove batch dim

#plt.subplot(2,2,3)
#plt.imshow(output_transforms(test_single))
axs[1,0].imshow(output_transforms(test_single))
#plt.title('input test image')

#plt.subplot(2,2,4)
#plt.imshow(mask,cmap='gray')
#plt.title('output mask')
axs[1,1].imshow(mask,cmap='gray')
plt.show()


# Saving the models

In [ ]:
pickle.dump(model, open('sem_seg_ode_model_neuclie', 'wb'))

# Brain Tumor Segmentation Dataset Downloading

In [ ]:
from google.colab import files
files.upload()

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation

!unzip -q '/content/lgg-mri-segmentation.zip' #-d '/content/brain_tumor_segmentation'
!rm  -r '/content/lgg-mri-segmentation.zip'
!rm  -r '/content/lgg-mri-segmentation'
!rm '/content/kaggle_3m/data.csv'
!rm '/content/kaggle_3m/README.md'

# Dataloader Brain Tumor Segmentation

In [ ]:
train_img=cv2.cvtColor(cv2.imread('/content/kaggle_3m/TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_11.tif'),cv2.COLOR_BGR2RGB)
plt.imshow(train_img)
plt.title('original image')
plt.show()


mask_image=cv2.cvtColor(cv2.imread('/content/kaggle_3m/TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_11_mask.tif'),cv2.COLOR_BGR2RGB)
plt.imshow(mask_image)
plt.title('mask image')
plt.show()


In [ ]:
train_ids=[]

for p in os.listdir('/content/kaggle_3m/'):
  for c in os.listdir('/content/kaggle_3m/'+p):
    #print(c)
    pth="/content/kaggle_3m/"+p+"/"+c
    #print(pth)
    if "mask" not in pth:
      #print(pth)
      train_ids.append(pth)

#print(len(train_ids))
train_ids=train_ids[:int(len(train_ids)*.20)]
train_ids,val_ids=train_test_split(train_ids,test_size=.2,random_state=42)
print(len(train_ids))
print(len(val_ids))

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, img_ids,im_shape, transform=None,mask_transform=None):
        self.transform = transform
        self.img_ids=img_ids
        self.im_shape=im_shape
        self.mask_transform=mask_transform

    def __len__(self):      
        return len(self.img_ids)

    def __getitem__(self, idx):
        #print(self.img_ids[idx])      
        image = cv2.imread(self.img_ids[idx])
        #print(image)
        #target_img_pth=img_ids[idx][:-4]+'_mask.tif'
        target_img = cv2.imread(self.img_ids[idx][:-4]+'_mask.tif',cv2.IMREAD_GRAYSCALE)

        if self.transform:
          image=self.transform(image)
        if self.mask_transform:
          target_img=self.mask_transform(target_img)

        return [image, target_img]


trans = transforms.Compose([
    Rescale(256),
    transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])


train_set = BrainTumorDataset(train_ids,256,trans,trans)
val_set = BrainTumorDataset(val_ids,256,trans,trans)

image_datasets = {
    'train': train_set, 'val': val_set
}

batch_size = 4

brain_tumor_dataloader = {
    'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)
}

In [ ]:
image,mask=next(iter(brain_tumor_dataloader['val']))
print(image.shape,mask.shape)
from torchvision.transforms import transforms, ToTensor, ToPILImage
def reverse_transform(inp,mask):   
    output_transforms = transforms.Compose([
            ToPILImage()])
    mask=output_transforms(mask) 
    inp=output_transforms(inp)
   
    return inp,mask

image,mask=reverse_transform(image[0],mask[0])
plt.imshow(image)
plt.title('input image')
plt.figure()
plt.imshow(mask,cmap='gray')
plt.title('target mask')


# Ode model for brain tumor segmentation

In [ ]:
model = SEM_SEG_ODE(ode,n_class=1,input_shape=(256,256,3))
if use_cuda:
    model = model.cuda()

summary(model, (3, 256, 256))

# Defining The metrics function

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
from loss import dice_loss
ce=nn.CrossEntropyLoss()
def dice_loss1(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Soft dice loss = 2*|A∩B| / |A|+|B|
    """
    numerator = 2 * (input * target).sum((1, 2, 3))
    denominator = (input + target).sum((1, 2, 3))

    return (1 - numerator / denominator).mean()

def dice_cof(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """Soft dice loss = 2*|A∩B| / |A|+|B|
    """
    numerator = 2 * (input * target).sum((1, 2, 3))
    denominator = (input + target).sum((1, 2, 3))

    return ( numerator / denominator).mean()


    
def calc_metrics(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss1(pred, target)
    dice_co=dice_cof(pred,target)

    loss = bce  + dice #* (1 - bce_weight)

    numerator =  (pred * target).sum((1, 2, 3))
    denominator = (pred + target).sum((1, 2, 3))

    iou=(numerator / (denominator-numerator)).mean()


    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] +=iou.data.cpu().numpy() * target.size(0)
    metrics['dice_co']+=dice_co.data.cpu().numpy() * target.size(0)


    return loss



# Defnining Training function

In [ ]:

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    best_iou=-1
    best_dice=1e10
    best_bce=1e10
    best_dice_co=-1



    for epoch in range(num_epochs):
        for_once=1
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in brain_tumor_dataloader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #inputs=inputs.type(torch.cuda.FloatTensor)
                #labels=labels.type(torch.cuda.FloatTensor)
                #print(inputs.shape,labels.shape)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    # if for_once==1:

                    #   img,segmented_mask=reverse_transform(inputs[0],outputs[0])
                    #   img,mask=reverse_transform(inputs[0],labels[0])
                    #   plt.imshow(img)
                    #   plt.title('input image')
                    #   plt.figure()
                    #   plt.imshow(mask,cmap='gray')
                    #   plt.title('target mask')

                    #   plt.figure()
                    #   plt.imshow(segmented_mask,cmap='gray')
                    #   plt.title('predected mask')
                    #   for_once=0

                    #print(outputs.shape)
                    #outputs=outputs.int()
                    loss = calc_metrics(outputs, labels, metrics)

                    #print(loss.item())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_dice = metrics['dice'] / epoch_samples
            epoch_bce = metrics['bce'] / epoch_samples
            epoch_iou = metrics['iou'] / epoch_samples
            epoch_dice_co=metrics['dice_co'] / epoch_samples

            # deep copy the model
            if phase == 'val':
              if epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
              if epoch_dice<best_dice:
                best_dice=epoch_dice
              if epoch_bce<best_bce:
                best_bce=epoch_bce
              if epoch_iou>best_iou:
                best_iou=epoch_iou
              if epoch_dice_co>best_dice_co:
                best_dice_co=epoch_dice_co

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        #for_once=0

    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val dice: {:4f}'.format(best_dice))
    print('Best val bce: {:4f}'.format(best_bce))
    print('Best val iou: {:4f}'.format(best_iou))
    print('Best val dice_co: {:4f}'.format(best_dice_co))
     # load best model weights
    model.load_state_dict(best_model_wts)
    return model



# Training The ODE model

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=150, gamma=0.1)

ode_brain_model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=200)

# Defining and training the author's(buda) model

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return self.conv(dec1)

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )
authors_brain_model=UNet()

if use_cuda:
    authors_brain_model = authors_brain_model.cuda()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, authors_brain_model.parameters()), lr=1e-3)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
print(summary(authors_brain_model, (3, 256, 256)))
authors_brain_model = train_model(authors_brain_model, optimizer_ft, exp_lr_scheduler, num_epochs=50)


# training fcn unet

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fcn_unet_brain = ResNetUNet(n_class=1)
fcn_unet_brain = fcn_unet_brain.to(device)

# check keras-like model summary using torchsummary
from torchsummary import summary
print(summary(fcn_unet_brain, input_size=(3, 192, 192)))

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, fcn_unet_brain.parameters()), lr=1e-3)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

fcn_unet_brain = train_model(fcn_unet_brain, optimizer_ft, exp_lr_scheduler, num_epochs=50)

# Predicting on validation data

In [ ]:
model=ode_brain_model

In [ ]:
image,mask=next(iter(brain_tumor_dataloader['val']))
image=image.to(device)
#print(image[1].shape)
mask=mask.to(device)
model=model.eval()
outputs=model(image)
outputs=F.sigmoid(outputs)

def reverse_transform(inp,mask):   
    output_transforms = transforms.Compose([
            ToPILImage()])
    mask=mask.cpu()
    inp=inp.cpu()
    mask=output_transforms(mask) 
    inp=output_transforms(inp)
   
    return inp,mask
fig, axs = plt.subplots(nrows=2,ncols=3, sharex='all', sharey='all', figsize=( 10,  6))

img,segmented_mask=reverse_transform(image[0],outputs[0])
img,msk=reverse_transform(image[0],mask[0])

axs[0,0].imshow(img)
#plt.title('input image')
#plt.figure()
axs[0,1].imshow(msk,cmap='gray')
#plt.title('target mask')

#plt.figure()
axs[0,2].imshow(segmented_mask,cmap='gray')
#plt.title('predected mask')

img,segmented_mask=reverse_transform(image[1],outputs[1])
#print(image[1].shape)
img,msk=reverse_transform(image[1],mask[1])

axs[1,0].imshow(img)
#plt.title('input image')
#plt.figure()
axs[1,1].imshow(msk,cmap='gray')
#plt.title('target mask')

#plt.figure()
axs[1,2].imshow(segmented_mask,cmap='gray')
#plt.title('predected mask')
